In [33]:
#This notebook focuses on transforming the data by adding new features that are more relevant and adding a "Failure"
#column that marks any date 14 days before a failure as a failure.

import pandas as pd
import matplotlib.pyplot as plt
import ast
from pandas.io.json import json_normalize

In [13]:
vehicles = pd.read_csv('vehicle_list.csv')
vehicles.head(10)

,fleet,truck_number,platform_id,vehicle_year,vehicle_make,vehicle_model,date_of_service,dpf_failure
0,Conway,432 3619,309450,2014,Freightliner,Cascadia,1/4/2019,Yes
1,Conway,432 3577,308374,2014,Freightliner,Cascadia,1/9/2019,Yes
2,Conway,426 6337,10315963,2013,Freightliner,Cascadia,1/17/2019,Yes
3,Conway,532 4279,10314364,2014,Freightliner,Cascadia,1/28/2019,Yes
4,Conway,432 3754,307112,2014,Freightliner,Cascadia,2/13/2019,Yes
5,Conway,532 5458,10312983,2015,Freightliner,Cascadia,3/16/2019,Yes
6,Conway,432 3874,308049,2014,Freightliner,Cascadia,3/19/2019,Yes
7,Conway,632 3472,310201,2013,Freightliner,Cascadia,4/9/2019,Yes
8,Conway,432 3721,303576,2014,Freightliner,Cascadia,4/17/2019,Yes
9,Conway,432 2497,302332,2012,Freightliner,Cascadia,4/20/2019,Yes


In [34]:
#vehicles = pd.read_csv('vehicle_list.csv')
unseenvehicles = pd.read_csv('unseen_vehicle_list.csv')
unseenvehicles.head(10)
unseenvehicles = unseenvehicles.drop(['Unnamed: 8'], axis = 1)

In [35]:
all_data = pd.DataFrame()

for i, e in unseenvehicles.iterrows():
    try:
        tmp = pd.read_csv('{}_daily_data.csv'.format(e.platform_id))
        thelist = tmp['distance_miles'].tolist() 
        templist = tmp['vehicle_serviced'].tolist()
        try:
            indexoffailure = templist.index(1)
            for i in range(indexoffailure-14, indexoffailure +1):
                templist[i] = "1"   #Any day 14 days before a service date would be marked as a failure
        except:
            None
        tmp['Failure'] = templist
        if len(all_data) == 0:  # first csv read replaces empty dataframe
            all_data = tmp
        else:  # all other csv reads append to dataframe
            all_data = all_data.append(tmp)
    except:
        None


all_data['date'] = pd.to_datetime(all_data['date'], infer_datetime_format=True)



In [38]:
#Percentage columns
all_data['percent_idle'] = all_data['idle_duration_mins']/all_data['duration_mins']
all_data['percent_fuel_lost'] = all_data['fuel_loss_gallons']/all_data['fuel_used_gallons']
all_data['percent_regen_inhibited'] = all_data['dpf_regen_inhibited_duration_mins']/all_data['duration_mins']
all_data['percent_regen_needed'] = all_data['dpf_regen_needed_duration_mins']/all_data['duration_mins']
all_data['percent_regen_inhibit_switch_not_active'] = all_data['dpf_regen_inhibit_switch_not_active_duration_mins']/all_data['duration_mins']

#New Columns
all_data['fuels_used_per_mile'] = all_data['fuel_used_gallons']/all_data['distance_miles']
all_data['miles_per_minutes'] = all_data['distance_miles']/all_data['duration_mins']
all_data['mins_idle_per_mile'] = all_data['idle_duration_mins']/all_data['distance_miles']
all_data['fuel_lost_per_mile'] = all_data['fuel_loss_gallons']/all_data['distance_miles']
all_data['min_regen_inhibited_per_mile'] = all_data['dpf_regen_inhibited_duration_mins']/all_data['distance_miles']
all_data['min_regen_needed_per_mile'] = all_data['dpf_regen_needed_duration_mins']/all_data['distance_miles']
all_data['min_regen_inhibit_switch_not_active_per_mile'] = all_data['dpf_regen_inhibit_switch_not_active_duration_mins']/all_data['distance_miles']

In [40]:
#all_data.to_csv('UnseenFeatures14days.csv')